In [ ]:
import os
import qianfan
import re
import pickle
import logging
from qianfan.utils import enable_log
enable_log(logging.ERROR)

with open('ernie_key.pkl','rb') as ernie_file:
    ernie_keys = pickle.load(ernie_file)

for keys in ernie_keys:
    os.environ[keys]=ernie_keys[keys]

In [ ]:
chat_comp = qianfan.ChatCompletion()

# 下面是一个与用户对话的例子
msgs = []
#计算tokens数
tokens_use =0
while True:
    if len(msgs)>8:
        for i in range(4):
            msgs.pop(0)
    myinput=input("输入：")
    ques_message={'role':'user', 'content':myinput}
    
    #停止：
    if re.sub(r'[ .。！!?？-]','', myinput) in {'拜拜', '再见','byebye', 'Byebye' ,'掰掰','','\n'}:
        break
    msgs.append(ques_message)
    print('用户：',myinput)
    #两个模型价格一样，但是输出长度不同
    model = 'ERNIE-Bot'
    model = 'ERNIE-3.5-4K-0205'
    # request_timeout:请求超时阙值, enable_citation:上角标注明来源,  max_output_tokens:最大tockens(messages所有加起来)
    resp = chat_comp.do(messages=msgs, model=model, stream=True, 
                        request_timeout=5, enable_citation=True, max_output_tokens=1024)
    
    answer=''
    print('Ernie Bot：',end='')
    for chunk in resp:
        answer+= chunk['result']
        print(chunk['result'], end='', flush=True)
    print('####end',end='', flush=True)
    answer_message = {'role':'assistant', 'content':answer}
    msgs.append(answer_message)
    tokens_use += chunk['usage']['total_tokens']